In [1]:
import math
import numpy as np
import pandas as pd
from datetime import datetime


In [2]:
#custom_date_parser = lambda x: datetime.strptime(x, "%d-%m-%y")
#df = pd.read_csv('data/data_6.csv',
#                 parse_dates=['date'],
#                date_parser=custom_date_parser)

In [3]:
df = pd.read_csv('Datasets/Southampton_vessels_Fixed_Formating.csv')# Write your code for importing the csv file here



In [4]:
#df['Recording']= pd.to_datetime(df['Recording'], infer_datetime_format=True)

In [5]:
df['ETA']= pd.to_datetime(df['ETA'],format="%d-%m-%y %H:%M")
df['ETD']= pd.to_datetime(df['ETD'],format="%d-%m-%y %H:%M")
df['Recording']= pd.to_datetime(df['Recording'], format="%d-%m-%y")
df = df.sort_values(by="Recording")

# Droping Useless Phases 

In [6]:
# Get indexes with thse value 
drop_indexNames = df[(df['Phase'] == 'CANCELED') | (df['Phase'] == 'CLOSED')  ].index 

# Delete these row indexes from dataFrame
df.drop(drop_indexNames , inplace=True)

df.sort_values(['Vessel Name', 'Recording'], ascending=[True, True], inplace=True)

In [7]:
#vessel_names_arr=pd.unique(df['Vessel Name'])
#temp_df=df[df['Vessel Name']=='NYK WREN']

In [8]:
# Converting All Phase expect Inbound to Arrived and Sorting 
df['Phase'] = df['Phase'].replace(['ARRIVED','COMPLETE','DEPARTED','WORKING'],'ARRIVED')
df['Ship_ID']=df['Vessel Name'] +' - '+ df['Ref']
df.sort_values(['Ship_ID', 'Recording'], ascending=[True, True], inplace=True)

# Droping Duplicates

In [9]:
#df.drop_duplicates(subset=['Ship_ID', 'ETA','Phase','Recording'] , keep ='first', inplace=True)

prev_phase='sasd'
prev_vessel='sasd'

for index, row in df.iterrows():

    if prev_vessel == row['Ship_ID'] and prev_phase == row['Phase']:
        df.drop(index , inplace=True)

    prev_vessel=row['Ship_ID']
    prev_phase=row['Phase']

In [10]:
#Sorting and Resting Index

df.sort_values(['Ship_ID', 'Recording'], ascending=[True, True], inplace=True)
df.reset_index(drop=True,inplace=True)

# Droping Value with No Arrived

In [11]:
prev_phase='sasd'
prev_vessel='sasd'
prev_index='888888'
for index, row in df.iterrows():
    if (prev_vessel != row['Ship_ID'] and prev_phase=='INBOUND'):
        df.drop(prev_index , inplace=True)


    prev_vessel=row['Ship_ID']
    prev_phase=row['Phase']
    prev_index=index

# Droping Value with No Inbound

In [12]:
prev_vessel='sasd'
for index, row in df.iterrows():
    if ( prev_vessel != row['Ship_ID'] and  (row['Phase'] != 'INBOUND')):
        df.drop(index , inplace=True)
    
    prev_vessel=row['Ship_ID']
    
df.sort_values(['Ship_ID', 'Recording'], ascending=[True, True], inplace=True)

df.reset_index(drop=True,inplace=True)
#df.drop(1596 , inplace=True)
df.reset_index(drop=True,inplace=True)


In [13]:
df.to_csv(r'Datasets/No_Duplicates_Kept_First.csv', index = False, header = True)

# Creating ATA Colums

In [ ]:
df['ATA']=np.nan
num_rows = df.shape[0]
i = 0
while i<num_rows:
    if df.iloc[i,2]=='INBOUND' and df.iloc[i+1,2] =='ARRIVED':
        df.iloc[i,12]= df.iloc[i+1,3]

    i+=2


In [ ]:
df.dropna(inplace=True)
df['Phase'] = df['Phase'].replace(['INBOUND'],'ARRIVED')
df = df[['Ship_ID','Vessel Name', 'Service', 'Phase', 'ETA', 'ATA', 'Berth', 'Side to', 'ETD', 'Next port of call', 'Ref', 'Recording', 'SN_No.']] #Reording Columns
df['ATA']= pd.to_datetime(df['ATA'],format="%d-%m-%y %H:%M")

In [ ]:
df['Delay'] =  df['ATA'] - df['ETA']


df['Delay']=df['Delay'].astype('timedelta64[m]')

In [ ]:
df.info()

In [ ]:
df['Vessel Name'].nunique()



In [ ]:
tempdf=df[df.duplicated(['Ref'], keep=False)]


In [ ]:
df.to_csv(r'Datasets/Vessels_ETA_ATA.csv', index = False, header = True)